In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
import plotly.graph_objs as go

In [3]:
print(__version__)

3.5.0


In [4]:
import pandas as pd

In [5]:
zetus = pd.read_csv("https://raw.githubusercontent.com/gautsi/gen-purpose-repo/master/zetus.csv")

In [6]:
zetus["time_"] = pd.to_datetime(zetus.time)

In [7]:
zetus.head()

,time,dur,type,time_
0,01/18/19 10:00,15,R,2019-01-18 10:00:00
1,01/18/19 10:00,10,L,2019-01-18 10:00:00
2,01/18/19 12:15,15,R,2019-01-18 12:15:00
3,01/18/19 13:00,0,S,2019-01-18 13:00:00
4,01/18/19 14:35,10,L,2019-01-18 14:35:00


In [8]:
init_notebook_mode(connected=True)

In [9]:
zetus.dtypes

time             object
dur               int64
type             object
time_    datetime64[ns]
dtype: object

In [10]:
import colorlover as cl

In [11]:
colorscale = cl.scales['5']['qual']['Dark2']

In [12]:
feed_data = [
    go.Bar(
        x = zetus[zetus["type"] == side].sort_values("time_").time_,
        y = 2 * (i - 0.5) * zetus[zetus["type"] == side].sort_values("time_").dur,
        name = side,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i]}) for i, side in enumerate(["L", "R"])
]

output_data = [
    go.Bar(
        x = zetus[zetus["type"] == output].sort_values("time_").time_,
        y = len(zetus[zetus["type"] == output]) * [60],
        name = output,
        base = -30,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i + 2]}) for i, output in enumerate(["S", "W"])
]

layout = go.Layout()

fig = go.Figure(data = feed_data + output_data, layout = layout)

iplot(fig)

In [13]:
plot(fig)

'file:///home/gautam_sisodia/gen-purpose-repo/zetus/temp-plot.html'

In [16]:
zetus["day"] = zetus.time_.dt.date

In [19]:
zetus["mod_dur"] = zetus.apply(lambda x: x.dur if x["type"] in ["R", "L"] else 1, axis = 1)

In [22]:
zetus.head()

,time,dur,type,time_,day,mod_dur
0,01/18/19 10:00,15,R,2019-01-18 10:00:00,2019-01-18,15
1,01/18/19 10:00,10,L,2019-01-18 10:00:00,2019-01-18,10
2,01/18/19 12:15,15,R,2019-01-18 12:15:00,2019-01-18,15
3,01/18/19 13:00,0,S,2019-01-18 13:00:00,2019-01-18,1
4,01/18/19 14:35,10,L,2019-01-18 14:35:00,2019-01-18,10


In [27]:
zetus_stats = zetus.groupby(["day", "type"], as_index = False)["mod_dur"].sum().sort_values("day").pivot(index = "day", columns = "type", values = "mod_dur")

In [29]:
zetus_num_feeds = zetus[zetus["type"].isin(["L", "R"])].groupby(["day", "type"], as_index = False)["mod_dur"].count().sort_values("day").pivot(index = "day", columns = "type", values = "mod_dur")

In [30]:
zetus_num_feeds

type,L,R
day,,
2019-01-18,4,4
2019-01-19,9,11
2019-01-20,8,9
2019-01-21,5,7


In [28]:
zetus_stats

type,L,R,S,W
day,,,,
2019-01-18,55,75,1,1
2019-01-19,147,160,1,5
2019-01-20,145,120,4,1
2019-01-21,40,90,2,4


In [33]:
zetus_stats.reset_index()

type,day,L,R,S,W
0,2019-01-18,55,75,1,1
1,2019-01-19,147,160,1,5
2,2019-01-20,145,120,4,1
3,2019-01-21,40,90,2,4


In [34]:
zetus_stats_2 = zetus_stats.reset_index().merge(
    right = zetus_num_feeds.rename(columns = {"L": "num_L", "R": "num_R"}).reset_index(),
    on = ["day"],
    how = "left")

In [36]:
zetus_stats_2["avg_L"] = zetus_stats_2.L / zetus_stats_2.num_L
zetus_stats_2["avg_R"] = zetus_stats_2.R / zetus_stats_2.num_R

In [37]:
zetus_stats_2

type,day,L,R,S,W,num_L,num_R,avg_L,avg_R
0,2019-01-18,55,75,1,1,4,4,13.750000,18.750000
1,2019-01-19,147,160,1,5,9,11,16.333333,14.545455
2,2019-01-20,145,120,4,1,8,9,18.125000,13.333333
3,2019-01-21,40,90,2,4,5,7,8.000000,12.857143
